 # Setup Workspace and Experiment

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

run = exp.start_logging()

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "compute-cluster-analytics"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute = ComputeTarget(ws, cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute = ComputeTarget.create(ws, cluster_name, compute_config)
    print("Created a new cluster")

compute.wait_for_completion()
print("Cluster ready")

Created a new cluster
Provisioning operation finished, operation "Succeeded"
Cluster ready


 # Hyperparameter Tuning

In [3]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C": uniform(0.01, 0.99)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=50, slack_factor=0.3)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(".", 
    script="train.py", 
    compute_target=compute, 
    environment=sklearn_env, 
    arguments=["--max_iter", 200])

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
    hyperparameter_sampling=ps,
    policy=policy, 
    primary_metric_name="Accuracy", 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=20)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

run = exp.submit(hyperdrive_config)

In [22]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [14]:
import joblib
# Get your best run and save the model from that run.
run.wait_for_completion()
best_run = run.get_best_run_by_primary_metric()
best_run.get_details()

{'runId': 'HD_3a970d65-5c9e-4e14-9430-b19783e461b5_17',
 'target': 'compute-cluster-analytics',
 'status': 'Completed',
 'startTimeUtc': '2022-03-07T12:21:20.612247Z',
 'endTimeUtc': '2022-03-07T12:23:54.035615Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '40adab49-6b73-415c-922f-f6afecadbe46',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--max_iter', '200', '--C', '0.7831171856027681'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'compute-cluster-analytics',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credenti

In [12]:
# best_run.register_model(model_name="best_random_sample_c_logistic_regression", model_path="outputs/model.pkl")
#
# to save the best model it would need to be an output of the training script

 # Auto ML

In [15]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path)

In [16]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
data = x.join(y)
data

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,1,0,0,1,7,1,116,1,999,...,0,1,0,0,0,0,0,0,0,0
32946,37,1,0,0,1,7,5,69,7,999,...,0,0,0,0,0,0,0,1,0,0
32947,26,0,0,0,0,5,2,135,4,999,...,0,0,0,0,0,0,0,1,0,0
32948,31,0,0,0,0,4,1,386,1,999,...,0,0,0,1,0,0,0,0,0,0


In [17]:
import os 
local_dir = "data"
os.makedirs(local_dir, exist_ok=True)
file_name = "cleaned_data.jsonl"
data.to_json(f"./{local_dir}/{file_name}", orient="records", lines=True)

In [18]:
from azureml.core import Dataset
# register dataset
datastore = ws.get_default_datastore()
datastore_path = "/users/mdelwig/udacity/project1/data"
datastore.upload("data", datastore_path, overwrite=True)
dataset = Dataset.Tabular.from_json_lines_files((datastore, f"{datastore_path}/{file_name}"))
dataset_name = "udacity_project1_data"
dataset.register(ws, dataset_name)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 3 files
Uploading data/.amlignore
Uploaded data/.amlignore, 1 files out of an estimated total of 3
Uploading data/.amlignore.amltmp
Uploaded data/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading data/cleaned_data.jsonl
Uploaded data/cleaned_data.jsonl, 3 files out of an estimated total of 3
Uploaded 3 files


{
  "source": [
    "('workspaceblobstore', 'users/mdelwig/udacity/project1/data/cleaned_data.jsonl')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseJsonLines",
    "DropColumns"
  ],
  "registration": {
    "id": "3ead13e2-d01b-4c3d-936d-db78a5ab9421",
    "name": "udacity_project1_data",
    "version": 1,
    "workspace": "Workspace.create(name='analytics-aml-ws', subscription_id='67021087-2c78-4338-bc3d-dd21237069f1', resource_group='analytics-mlops-rg')"
  }
}

In [19]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute)

In [20]:
# Submit your automl run

ml_run = exp.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_bfc9df2d-fc8b-432e-8851-2a825a8dbbb0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
RunDetails(ml_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [26]:
# Retrieve and save your best automl model.
ml_run.wait_for_completion()
best_ml_run = ml_run.get_best_child()
best_ml_run.get_details()

{'runId': 'AutoML_bfc9df2d-fc8b-432e-8851-2a825a8dbbb0_30',
 'target': 'compute-cluster-analytics',
 'status': 'Completed',
 'startTimeUtc': '2022-03-07T14:11:48.794672Z',
 'endTimeUtc': '2022-03-07T14:11:52.549516Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'compute-cluster-analytics\',\'subscription_id\':\'67021087-2c78-4338-bc3d-dd21237069f1\',\'region\':\'westeurope\',\'spark_service\':None}","ensemble_run_id":"AutoML_bfc9df2d-fc8b-432e-8851-2a825a8dbbb0_30","experiment_name":"udacity-project","workspace_name":"analytics-aml-ws","subscr

In [27]:
best_ml_run.register_model(model_name="udacity1_best_ml_model", model_path="outputs/model.pkl")

Model(workspace=Workspace.create(name='analytics-aml-ws', subscription_id='67021087-2c78-4338-bc3d-dd21237069f1', resource_group='analytics-mlops-rg'), name=udacity1_best_ml_model, id=udacity1_best_ml_model:2, version=2, tags={}, properties={})

In [28]:
try:
    compute = ComputeTarget(ws, cluster_name)
    compute.delete()
    print("Deleted cluster")
except ComputeTargetException:
    print("Cluster allready deleted")

Deleted cluster
Current provisioning state of AmlCompute is "Deleting"

